In [1]:
import math

In [2]:
class Node:
    def __init__(self, data, children=None):
        self.data = data
        self.children = children if children is not None else []

    def has_children(self):
        return bool(self.children)

    def children(self):
        return self.children

In [3]:
def get_subsets(nodes, size):
    return chain.from_iterable(combinations(nodes, r) for r in range(1, size+1))

def min_weight(nodes):
    return min(sum(weight for _, _, weight in get_neighbors(u, v)) for u, v in combinations(nodes, 2))

def get_neighbors(u, v, graph):
    return [(u, v, weight) for u, v, weight in graph if (u, v) in [(u, v), (v, u)]]

def max_weight(node, graph):
    return max(weight for _, _, weight in get_neighbors(node, graph))

In [4]:
def is_feasible(nodes):
    return is_connected(nodes)


In [5]:
def get_maximum_possible_weight(nodes):
    return sum(max_weight(node) for node in nodes)


In [6]:
def is_connected(network, nodes):
    visited = set()
    queue = list(nodes)
    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            for neighbor in get_neighbors(network, node):
                if neighbor in nodes and neighbor not in visited:
                    queue.append(neighbor)
    return len(visited) == len(nodes)


In [7]:
def get_minimum_possible_weight(nodes):
    return sum(min_weight(node) for node in nodes)


In [8]:
def PARTITIONINGS(S0):
    partitionings = []
    for subset in get_subsets(S0):
        partitionings.append(subset)
    return partitionings


In [9]:
def LINEAR_SWEEP(T, S0, S1):
    # Initialize the lower and upper bounds for the linear search
    lower_bound = 0
    upper_bound = get_maximum_possible_weight(S0 | S1)
    
    # Perform the linear search
    for value in range(lower_bound, upper_bound + 1):
        if is_feasible(T, S0 | S1, value):
            return value
    
    # Return the optimal value found by the linear search
    return upper_bound


In [10]:
def BINARY_SEARCH(T, S0):
    # Initialize the lower and upper bounds for the binary search
    lower_bound = 0
    upper_bound = get_maximum_possible_weight(S0)
    
    # Perform the binary search
    while lower_bound <= upper_bound:
        mid = (lower_bound + upper_bound) // 2
        if is_feasible(T, S0, mid):
            upper_bound = mid - 1
        else:
            lower_bound = mid + 1
    
    # Return the optimal value found by the binary search
    return lower_bound


In [11]:
def EXPANDABLE_NODES(T):
    # Initialize an empty set to store the expandable nodes
    expandable_nodes = set()
    
    # Iterate through each node in the tree
    for node in T:
        # If the node has children, add them to the set of expandable nodes
        if node.has_children():
            expandable_nodes |= set(node.children)
    
    return expandable_nodes


In [12]:
def fU(T):
    # Initialize the upper bound to 0
    upper_bound = 0
    
    # Calculate the upper bound by adding the maximum possible weight
    # for each pair of nodes in T that are not directly connected
    for node1 in T:
        for node2 in T:
            if node1 != node2 and not is_connected(node1, node2):
                upper_bound += get_maximum_possible_weight(node1, node2)
                
    return upper_bound


In [13]:
def fL(T):
    # Initialize the lower bound to 0
    lower_bound = 0
    
    # Calculate the lower bound by adding the minimum possible weight
    # for each pair of nodes in T that are not directly connected
    for node1 in T:
        for node2 in T:
            if node1 != node2 and not is_connected(node1, node2):
                lower_bound += get_minimum_possible_weight(node1, node2)
                
    return lower_bound


In [23]:
def NETWORK_OPT(T, M):
    global N, c
    if fL(T) >= fL(T) + c or fU(T) <= fL(T) - c:
        return N,c
    S0, S1, S2 = EXPANDABLE_NODES(T)
    if not S0:
        N, c = T, abs(f(T) - f(T))
    elif not S1 and len(S0) <= M:
        T0 = T | BINARY_SEARCH(T, S0) - S0
        NETWORK_OPT(T0, M)
    elif not S2 and max(len(S0), len(S1)) <= M:
        T0 = T | LINEAR_SWEEP(T, S0, S1) - S0 - S1
        NETWORK_OPT(T0, M)
    else:
        for P in PARTITIONINGS(S0):
            T0 = T | P - S0
            NETWORK_OPT(T0, M)

In [26]:
class TreeNode:
    def __init__(self, node_id, node_weight, children=None):
        self.node_id = node_id
        self.node_weight = node_weight
        self.children = children if children else []
        


In [ ]:
# Define the decomposition tree with a singular node
# Create the root node
root = TreeNode(node_id=1, node_weight=5)

# Create child nodes
child1 = TreeNode(node_id=2, node_weight=3)
child2 = TreeNode(node_id=3, node_weight=4)
child3 = TreeNode(node_id=4, node_weight=2)

# Add child nodes to the root node
root.children = [child1, child2, child3]

# Define the maximum subset size for tabulation
M = 4

# Define the desired network function
def fT(network):
    return sum(node.node_weight for node in network)

# Call the NETWORK-OPT function
network, cost = NETWORK_OPT(root, M)

# Print the minimum cost network
print("Minimum Cost Network:")
for node in network:
    print(node.node_id, node.node_weight)
print("Cost:", cost)